# Residual Momentum Strategy
### Kevin Tranchina 861785
### Giacomo De Gobbi 860913

In [199]:
import pandas_datareader.data as reader
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import math
import ast
from plotly import graph_objs as go
from plotly.subplots import make_subplots
from collections import Counter

In [200]:
directory = 'momentum/'

#### We are building a residuals momentum strategy based on the S&P500 index using price data from 01/03/19 to 31/03/21

# Scarico Dati

In [201]:
########## SCARICO DATI e salvataggio in csv

start = dt.datetime(2019,3,1)
end = dt.datetime(2021,3,31)



### stocks
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0] # for obtain the stocks tickers list
tickers = df.Symbol.to_list()
print(len(tickers)) # counting stocks

df = yf.download(tickers,start=start,end=end) # Yahoo finance as source
stocks = df['Adj Close']
stocks= stocks[1:-2]
stocks.to_csv("stocks.csv")

stocks_pct = stocks.pct_change() ## prices are converted in percentage  
stocks_pct
stocks_pct = stocks_pct[1:]


#### funds 

funds = ['SPY']
fundsret = reader.get_data_yahoo(funds,start,end)['Adj Close'].pct_change()
fundsret = fundsret[1:-1]
fundsret = fundsret.resample('M').agg(lambda x: (x+1).prod()-1) # this gives us monthly returnd
fundsretPrice = reader.get_data_yahoo(funds,start,end)['Adj Close']

fundsret.to_csv("fundsret.csv")



# Fama French
factors = reader.DataReader('F-F_Research_Data_Factors_daily','famafrench',start,end)[0]
factors = factors[1:-3]
factors = factors[['SMB','HML']]
factors.to_csv("factors.csv")


505
[**********************63%*****                  ]  320 of 505 completed

KeyboardInterrupt: 

# Lettura Dati salvati

In [202]:
stocks = pd.read_csv(directory + "stocks.csv")

[*********************100%***********************]  504 of 505 completed

In [203]:
stocks = stocks.set_index('Date')
stocks.index = pd.to_datetime(stocks.index)
stocks

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-01,79.782806,34.181717,158.647507,42.613480,69.983582,80.022308,348.059998,75.875458,158.558273,264.010010,...,51.708683,123.413208,70.288681,48.249748,74.463387,92.004433,123.320869,202.210007,47.938522,94.501099
2019-03-04,78.692719,33.273895,156.454773,42.827797,69.424423,77.611420,338.200012,75.904411,157.225677,258.160004,...,51.859337,119.680809,70.561050,47.817631,74.346375,91.782555,122.357178,201.729996,47.648102,94.619522
2019-03-05,78.309708,32.790375,155.378143,42.749863,68.943878,76.820648,333.209991,75.518234,158.046478,257.380005,...,51.689854,119.817230,70.455620,48.082794,74.287872,92.544678,122.042519,201.639999,47.582516,94.530701
2019-03-06,77.337471,32.326591,156.030029,42.503883,68.227440,73.898636,317.850006,75.016197,156.810455,256.399994,...,51.868748,118.267746,69.656082,46.894482,73.956375,93.191025,121.187004,201.240005,46.533245,92.941711
2019-03-07,77.317825,31.793739,152.237198,42.011913,68.358498,73.609337,317.420013,74.147285,156.289017,255.460007,...,52.273602,116.854691,70.429260,47.356052,73.439606,93.133148,119.367821,201.990005,45.633869,91.342842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-22,123.422195,23.830000,182.990005,123.389999,104.627876,116.349998,306.160004,121.025635,264.381775,452.410004,...,63.750000,125.349998,55.910000,62.336536,102.430000,108.750000,159.996414,463.769989,55.279999,155.398560
2021-03-23,121.794678,22.270000,179.529999,122.540001,103.580605,113.769997,298.619995,120.049217,265.209198,460.200012,...,65.440002,122.129997,55.220001,59.830696,100.400002,108.000000,155.902649,462.649994,53.580002,155.807938
2021-03-24,120.656403,21.809999,181.729996,120.089996,101.821983,115.370003,294.209991,117.588249,266.724518,451.510010,...,65.580002,119.959999,56.340000,60.249996,101.110001,107.080002,157.210648,463.809998,53.009998,155.198837


In [204]:
stocks_pct = stocks.pct_change() ## coverting in percentage
stocks_pct
stocks_pct = stocks_pct[1:]
stocks_pct_monthly = stocks_pct.resample('M').agg(lambda x: (x+1).prod()-1)
stocks_pct_monthly.index = pd.to_datetime(stocks_pct_monthly.index)
stocks_pct_monthly

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-31,-0.010586,-0.083141,0.062087,0.085615,0.006117,-0.041697,-0.179481,0.017178,0.071985,0.009394,...,0.030859,0.001184,0.010000,0.011171,0.034961,0.046555,0.020190,0.036200,-0.112566,0.051384
2019-04-30,-0.021392,0.076196,-0.024688,0.056436,-0.001829,-0.059859,-0.028642,-0.000681,0.046375,0.085407,...,0.005159,-0.052449,-0.006436,0.031055,0.055162,0.045887,-0.035552,0.007684,0.086325,0.013298
2019-05-31,-0.145860,-0.201036,-0.068062,-0.124213,-0.033757,0.046863,-0.055838,-0.043112,-0.025182,-0.063440,...,0.014867,-0.145617,-0.108356,0.053589,-0.107186,-0.015435,-0.074943,-0.188027,-0.121224,-0.007757
2019-06-30,0.113646,0.197576,-0.005160,0.130519,-0.052014,0.095042,-0.005460,0.104689,0.037626,0.087671,...,0.044604,0.152575,0.082803,0.085006,0.126920,0.081290,0.035611,0.221944,0.067564,0.123107
2019-07-31,-0.068397,-0.064397,-0.022707,0.076394,-0.069914,0.022167,0.069369,0.039550,0.042269,0.014288,...,0.002017,-0.031462,-0.029623,-0.066998,-0.040053,0.016716,0.147698,0.006683,-0.019791,0.013799
2019-08-31,0.024492,-0.134627,-0.084241,-0.016461,-0.013209,-0.051583,-0.306900,-0.020436,0.029027,-0.048016,...,0.077336,-0.085710,-0.067624,-0.042241,-0.042741,0.041618,0.030119,-0.027787,-0.080728,0.100357
2019-09-30,0.079945,0.025086,0.199453,0.072962,0.151810,0.000729,-0.078624,-0.019339,-0.029369,-0.029032,...,0.016945,-0.078416,0.031104,0.024168,0.039290,-0.028687,-0.012132,0.006536,0.083475,-0.014476
2019-10-31,-0.011484,0.114572,-0.017654,0.110684,0.081618,0.037046,0.166901,0.003289,-0.031886,0.006081,...,-0.021267,-0.053806,-0.043053,0.027575,-0.034065,-0.103324,0.006994,0.152638,0.088724,0.028044
2019-11-30,0.066271,-0.040813,-0.033235,0.077554,0.102828,0.034436,-0.054919,0.022007,0.084888,0.113698,...,-0.031806,0.026436,0.020447,0.032129,0.010692,-0.005968,0.051002,0.054946,0.033958,-0.057849


In [205]:
fundsret = pd.read_csv(directory+"fundsret.csv")
fundsret = fundsret.set_index('Date')
fundsret.index = pd.to_datetime(fundsret.index)
fundsret = fundsret[1:]

### fundsret mensile
fundsret_monthly = fundsret.resample('M').agg(lambda x: (x+1).prod()-1)
fundsret_monthly.index = pd.to_datetime(fundsret_monthly.index)


fundsret

,SPY
Date,
2019-03-04,-0.003638
2019-03-05,-0.001360
2019-03-06,-0.006057
2019-03-07,-0.008366
2019-03-08,-0.002000
...,...
2021-03-22,0.007985
2021-03-23,-0.007871
2021-03-24,-0.005083


### Fama-French 3 Factor Model
$ r_{i,t}= \alpha_i + \beta_{1,i}RM_t + \beta_{2,i}SMB_t + \beta_{3,i}HML_t + \epsilon_{i,t} $ <br>
<ul>
<li>RM: the return of the market portfolio</li>
<li>SMB: Small Minus Big (market capitalization)</li>
<li>HML: High Minus Low (book to market ratio)</li>
</ul>
They measure the historic excess returns of small caps over big caps and of value stocks over growth stocks.

In [206]:
factors = pd.read_csv(directory +'factors.csv')
factors = factors.set_index('Date')
factors.index = pd.to_datetime(factors.index)
factors = factors[['SMB','HML']]
factors

,SMB,HML
Date,,
2019-03-04,-0.42,0.37
2019-03-05,-0.31,-0.23
2019-03-06,-1.26,0.07
2019-03-07,-0.06,-0.32
2019-03-08,0.06,-0.17
...,...,...
2021-03-22,-1.15,-2.20
2021-03-23,-2.67,-1.05
2021-03-24,-1.98,1.46


In [207]:
fundsret3 = factors.join(fundsret)

#### Building the function to perform a rolling regression with a window=12

In [208]:
start = dt.datetime(2019,3,1)
end = dt.datetime(2021,3,31)

# Start window 
startWindow = start

# Close window
endWindow = start + relativedelta(months=+12) 

df = pd.DataFrame()

#### Rolling 

In [209]:
while(endWindow<=end):
    
    filtroData = (fundsret3.index <= endWindow + relativedelta(months=-1)) & (fundsret3.index >= startWindow)
    filtroStocksData = (stocks_pct.index <= endWindow + relativedelta(months=-1)) & (stocks_pct.index >= startWindow)
    fundsret2 = fundsret3[filtroData]
    stocks_pct2 = stocks_pct[filtroStocksData]
#     print("\n")
#     print("startWindow : " + str(startWindow))
#     print("endWindow : " + str(endWindow))
#     print("\n")
    
    # inizio nuova finestra
    startWindow = startWindow + relativedelta(months=+1)
    
    # fine nuova finestra
    endWindow = startWindow + relativedelta(months=+12)
    
    stocks_pct2.index = fundsret2.index
    
    x = fundsret2
    x
    
    y = stocks_pct2
    y
    
#    x = sm.add_constant(x)
#    x
    
    OLS_model = sm.OLS(y,x).fit() # training the model
    predicted_values = OLS_model.predict()  # predicted values
    residual_values = OLS_model.resid # residual values
    coefficient = OLS_model.params # residual values
    coefficient
    residual_values.index = pd.to_datetime(residual_values.index)
    
    # Extracting the residual to perform the momentum analysis
    
    residuals = residual_values.resample('12M').agg(lambda x: (x+1).prod()-1) # dà il rendimento mensile
    
    residualMomentum = residuals.iloc[0]
    residualMomentum
    
    residualMomentum = pd.DataFrame(residualMomentum)
    
    # Taking the best 2% of stocks to build our portfolio
    
    residualMomentum['cinquantile'] = pd.qcut(residualMomentum.iloc[:,0],50,labels=False,duplicates='drop')
    
    winners = residualMomentum[residualMomentum['cinquantile'] == 49]
    losers = residualMomentum[residualMomentum['cinquantile'] == 1]
    winners.index
    
    investedMonth = endWindow #+relativedelta(months=+1)
    investedMonth = investedMonth +relativedelta(days=-1)
    investedMonth = pd.to_datetime(investedMonth)
    
    stocks_winners = stocks_pct_monthly[winners.index]
    stocks_winners = stocks_winners[stocks_winners.index == investedMonth]
    stocks_winners['portafoglio'] = stocks_winners.iloc[-1].mean()
    
    stocks_winners.loc[:,'winners'] = str(list(winners.index))
    stocks_winners.loc[:,'winners']
    stocks_winners = stocks_winners[['winners','portafoglio']]
    
    df = df.append(stocks_winners)
    df

#### Calculating the portfolio return

In [210]:
# Normalizzazione
df.index = pd.to_datetime(df.index)
df['SPY'] = fundsret_monthly[fundsret_monthly.index.isin(df.index)]

rendimentoIndice = df['SPY'].agg(lambda x: (x+1).prod()-1) # d
print('rendimentoIndice : '+str(rendimentoIndice))
rendimentoPortafoglio = df['portafoglio'].agg(lambda x: (x+1).prod()-1) # d
rendimentoPortafoglio
print('rendimentoPortafoglio : '+str(rendimentoPortafoglio))

rendimentoIndice : 0.36601641391810524
rendimentoPortafoglio : 0.6734470883114303


### PLOT

In [226]:
spy = np.array(df['SPY'])
portafoglio = np.array(df['portafoglio'])

fig = make_subplots(rows=1, cols=2, column_widths=[1,1], print_grid=True, horizontal_spacing=1)
fig.data = []
fig.layout = {}

fig['layout']['xaxis']['title']='Mesi di Investimento'
fig['layout']['yaxis']['title']='Rendimento'
# import plotly.express as px
# df = px.data.tips()

# fig = px.scatter(df, x="total_bill", y="tip", color="sex",
#     labels=dict(total_bill="Total Bill ($)", tip="Tip ($)", sex="Payer Gender")
# )

# fig = go.Figure()

# fig.update_layout(
#     title="Plot Title",
#     xaxis_title="x Axis Title",
#     yaxis_title="y Axis Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="#7f7f7f"
#     )


fig.add_trace(go.Scatter(
    x=df.index,
    y=spy,
    name='SPY'
))

fig.add_trace(go.Scatter(
    x=df.index, 
    y=portafoglio,
    name='PORTAFOGLIO'
))

fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]



#### Comparing the portfolio risk with the index risk

In [224]:
### RISK

initial_weight = np.array([0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09])

start = dt.datetime(2020,3,1)
end = dt.datetime(2021,3,31)

# Inizio finestra
startWindow = start
# Fine finestra
endWindow = start + relativedelta(months=+12) 

seriesRisk = pd.DataFrame()

for index, row in df.iterrows():
    
    s = stocks_pct[ast.literal_eval(row['winners'])]
    filtroData = (s.index <= endWindow) & (s.index >= startWindow)
    
    s = s[filtroData]
    s
    
    matrix_covariance_portfolio = s 
    matrix_covariance_portfolio = (matrix_covariance_portfolio.cov())
    
    matrix_covariance_portfolio
    
    portfolio_variance = np.dot(initial_weight.T,np.dot(matrix_covariance_portfolio, initial_weight))
    
    #standard deviation (risk of portfolio)
    portfolio_risk = np.sqrt(portfolio_variance)*np.sqrt(22)
    portfolio_risk
    
    d = {'Date': [s.index[0]], 'riskMonthly': [portfolio_risk]}
    d = pd.DataFrame(data=d,index=None)
    d
    
    seriesRisk = seriesRisk.append(d)
    
    # inizio nuova finestra
    startWindow = startWindow + relativedelta(months=+1)
    # fine nuova finestra
    endWindow = startWindow + relativedelta(months=+12)
    
### Risk Spy

fundsret = fundsret[fundsret.index >'2019-12-31']

matrix_covariance_spy = fundsret #.iloc[:,:-1]
matrix_covariance_spy = (matrix_covariance_spy.cov())

#standard deviation (risk of portfolio)
risk_spy = np.sqrt(matrix_covariance_spy)*np.sqrt(252)

In [79]:
riskMonthly = seriesRisk['riskMonthly'].mean()
print('riskMonthly : '+ str(riskMonthly))
riskAnnualPortafoglio = riskMonthly*(np.sqrt(12))
print('riskAnnualPortafoglio : '+ str(riskAnnualPortafoglio))
print('risk_spy : '+ str(risk_spy.iloc[0,0]))

riskMonthly : 0.08317430701110537
riskAnnualPortafoglio : 0.2881242512551336
risk_spy : 0.3088973622530189


In [ ]:
riskMonthly : 0.08370661512901069
riskAnnualPortafoglio : 0.2899682206661203
risk_spy : 0.3088973622530189

In [131]:
lista = list()
for index, row in df.iterrows():
    lista.append(ast.literal_eval(row['winners']))

In [134]:
flat_list = [item for sublist in lista for item in sublist]
flat_list

['APA',
 'AVGO',
 'CAG',
 'CMG',
 'DVN',
 'KMX',
 'NVDA',
 'PVH',
 'PXD',
 'SJM',
 'ULTA',
 'DIS',
 'F',
 'HAS',
 'IR',
 'QCOM',
 'STZ',
 'TEL',
 'TER',
 'TWTR',
 'URI',
 'WYNN',
 'AIG',
 'CNC',
 'CZR',
 'ENPH',
 'LHX',
 'PEAK',
 'RCL',
 'RMD',
 'TTWO',
 'VTR',
 'WELL',
 'AAL',
 'APTV',
 'DVA',
 'DXCM',
 'EMN',
 'GNRC',
 'HFC',
 'IPGP',
 'NWS',
 'WDC',
 'WHR',
 'DFS',
 'ENPH',
 'EW',
 'FISV',
 'HAS',
 'MU',
 'TER',
 'TWTR',
 'UHS',
 'UPS',
 'ZBH',
 'BA',
 'ETN',
 'NCLH',
 'NWL',
 'OKE',
 'SYY',
 'TDG',
 'TGT',
 'TSN',
 'VTR',
 'WELL',
 'AAP',
 'ABBV',
 'DLTR',
 'DPZ',
 'ETSY',
 'IPGP',
 'LKQ',
 'MPC',
 'PRGO',
 'TPR',
 'TSLA',
 'AAL',
 'ABMD',
 'ALGN',
 'BIIB',
 'CI',
 'CNC',
 'GNRC',
 'KHC',
 'LEG',
 'MHK',
 'TSLA',
 'ADSK',
 'DISCA',
 'DISCK',
 'DVA',
 'DXC',
 'DXCM',
 'FTNT',
 'PAYC',
 'QRVO',
 'REGN',
 'SCHW',
 'AMD',
 'CAG',
 'DVN',
 'ENPH',
 'EOG',
 'FANG',
 'NEM',
 'PXD',
 'SWKS',
 'TSLA',
 'WDC',
 'APA',
 'DRI',
 'ENPH',
 'GE',
 'LB',
 'LEN',
 'PAYC',
 'PENN',
 'PHM',
 'TDG',
 

In [167]:
print("Number of stocks in wich we have invested : "+str(len(set(flat_list))))
s = Counter(flat_list)
s

Number of stocks in wich we have invested : 110


Counter({'APA': 2,
         'AVGO': 1,
         'CAG': 2,
         'CMG': 1,
         'DVN': 2,
         'KMX': 1,
         'NVDA': 2,
         'PVH': 1,
         'PXD': 2,
         'SJM': 1,
         'ULTA': 1,
         'DIS': 1,
         'F': 1,
         'HAS': 2,
         'IR': 1,
         'QCOM': 1,
         'STZ': 1,
         'TEL': 1,
         'TER': 2,
         'TWTR': 2,
         'URI': 1,
         'WYNN': 1,
         'AIG': 1,
         'CNC': 3,
         'CZR': 1,
         'ENPH': 5,
         'LHX': 1,
         'PEAK': 1,
         'RCL': 1,
         'RMD': 1,
         'TTWO': 1,
         'VTR': 2,
         'WELL': 2,
         'AAL': 2,
         'APTV': 1,
         'DVA': 2,
         'DXCM': 3,
         'EMN': 1,
         'GNRC': 2,
         'HFC': 1,
         'IPGP': 2,
         'NWS': 1,
         'WDC': 2,
         'WHR': 1,
         'DFS': 1,
         'EW': 1,
         'FISV': 1,
         'MU': 1,
         'UHS': 1,
         'UPS': 1,
         'ZBH': 1,
         'BA': 1,
   

In [173]:
{x: count for x, count in s.items() if count >= 2}

{'APA': 2,
 'CAG': 2,
 'DVN': 2,
 'NVDA': 2,
 'PXD': 2,
 'HAS': 2,
 'TER': 2,
 'TWTR': 2,
 'CNC': 3,
 'ENPH': 5,
 'VTR': 2,
 'WELL': 2,
 'AAL': 2,
 'DVA': 2,
 'DXCM': 3,
 'GNRC': 2,
 'IPGP': 2,
 'WDC': 2,
 'TDG': 2,
 'DPZ': 2,
 'ETSY': 2,
 'TSLA': 4,
 'BIIB': 2,
 'CI': 2,
 'PAYC': 2,
 'REGN': 2}